In [2]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt
import pickle
import momepy
# pip install azureml-opendatasets-runtimeusing
from azureml.opendatasets import NycTlcYellow

from datetime import datetime
from dateutil import parser


In [60]:
# Only need to run this once ever
def preprocess_lion():
    # Download data from https://www.dropbox.com/sh/927yoof5wq6ukeo/AAA--Iyb7UUDhfWIF2fncppba?dl=0
    # Put all files into 'data_unwrangled/LION' or change path below
    lion_filename = 'data_unwrangled/LION/links.shp'
    new_folder = 'data/infrastructure/'
    # Load all LION data
    links = gpd.read_file('data_unwrangled/LION/links.shp')
    # Only consider links in Manhattan
    links = links[links['LBoro']==1]
    # Only consider links that are normal streets
    links = links[links['FeatureTyp']=='0']
    # Only consider constructed links
    links = links[links['Status']=='2']
    # Ensure *undirected* graph is connected
    # Note: We could do this for directed graph but maximum size
    # of strongly connected component is 430
    graph = momepy.gdf_to_nx(links, approach="primal", directed=False)
    for component in nx.connected_components(graph):
        if len(component) > 10000:
            graph = graph.subgraph(component)
    # Use resulting links and nodes as infrastructure
    nodes, links = momepy.nx_to_gdf(graph)
    # Save both nodes and links
    nodes.to_file(new_folder + 'nodes.json', driver='GeoJSON')
    links.to_file(new_folder + 'links.json', driver='GeoJSON')

In [62]:
links = gpd.read_file('data/infrastructure/links.json')
nodes = gpd.read_file('data/infrastructure/nodes.json')